In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
HOME = '/content/drive/MyDrive/20240424'

import os
%cd {HOME}

/content/drive/MyDrive/20240424


In [ ]:
import cv2
import dlib
import numpy as np
from google.colab.patches import cv2_imshow


In [ ]:
!pip install ffmpeg-python

import ffmpeg
from base64 import b64encode
from IPython.display import HTML

def get_video_metadata(video_path):
    try:
        probe = ffmpeg.probe(video_path)
        video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
        return {
            'video_codec': video_stream['codec_name'] if video_stream else 'None',
        }
    except Exception as e:
        return {'error': str(e)}

def convert_video_to_h264(input_path, output_path):
    try:
        ffmpeg.input(input_path).output(output_path, vcodec='libx264').run(overwrite_output=True)
        return True
    except ffmpeg.Error as e:
        return False

def check_and_convert_video(video_path):
    metadata = get_video_metadata(video_path)

    if metadata.get('error'):
        return f"Error retrieving metadata: {metadata['error']}", 0

    if metadata['video_codec'] == 'mpeg4':
        output_path = video_path.replace('.mp4', '_converted.mp4')
        success = convert_video_to_h264(video_path, output_path)
        if success:
            return output_path, 1
        else:
            return "Failed to convert video.",0
    else:
        return video_path, 2

def play_video(video_path):

    play_video_file, result = check_and_convert_video(video_path)

    if result!=0:
        # ��������� ��������� ������ base64��� ���������
        try:
            with open(play_video_file, "rb") as video_file:
                video_encoded = b64encode(video_file.read()).decode('ascii')

            # HTML ������ ������
            video_html = f'''
            <video width=800 controls>
            <source src="data:video/mp4;base64,{video_encoded}" type="video/mp4">
            </video>
            '''
            display(HTML(video_html))
        except Exception as e:
            print(f"Error: {e}")
    else :
        print("unknown file!!!")

In [ ]:
detector = dlib.get_frontal_face_detector()

cap = cv2.VideoCapture('videos/girl.mp4')
sticker_img = cv2.imread('imgs/sticker01.png', cv2.IMREAD_UNCHANGED)

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')

out = cv2.VideoWriter('output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

while cap.isOpened():
  ret, img = cap.read()

  if not ret:
    break

  dets = detector(img)

  for det in dets:
    x1 = det.left()
    y1 = det.top()
    x2 = det.right()
    y2 = det.bottom()

    try:

      overlay_img = sticker_img.copy()
      overlay_img = cv2.resize(overlay_img, dsize=(x2-x1, y2-y1))
      overlay_alpha = overlay_img[:, :, 3:] / 255.0
      background_alpha = 1 - overlay_alpha
      img[y1:y2, x1:x2] = overlay_alpha * overlay_img[:, :, :3] + background_alpha * img[y1:y2, x1:x2]

      #cv2.rectangle(img, pt1 = (x1, y1), pt2 = (x2, y2), color = (255, 0, 0), thickness = 2)

    except:
      pass

  out.write(img)

out.release()

play_video('output.mp4')

In [15]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('models/shape_predictor_5_face_landmarks.dat')

cap = cv2.VideoCapture('videos/boy.mp4')
sticker_img = cv2.imread('imgs/glasses.png', cv2.IMREAD_UNCHANGED)

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')

out = cv2.VideoWriter('output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

while cap.isOpened():
  ret, img = cap.read()

  if not ret:
    break

  dets = detector(img)

  for det in dets:
    shape = predictor(img, det)

    x1 = det.left()
    y1 = det.top()
    x2 = det.right()
    y2 = det.bottom()

    try:
      glasses_x1 = shape.parts()[2].x - 20
      glasses_x2 = shape.parts()[0].x + 20

      h, w, c = sticker_img.shape

      glasses_w = glasses_x2 - glasses_x1
      glasses_h = int(h / w * glasses_w)

      center_y = (shape.parts()[2].y + shape.parts()[0].y) / 2

      glasses_y1 = int(center_y - glasses_h / 2)
      glasses_y2 = glasses_y1 + glasses_h


      overlay_img = sticker_img.copy()
      overlay_img = cv2.resize(overlay_img, dsize=(glasses_w, glasses_h))
      overlay_alpha = overlay_img[:, :, 3:] / 255.0
      background_alpha = 1 - overlay_alpha
      img[glasses_y1:glasses_y2, glasses_x1:glasses_x2] = overlay_alpha * overlay_img[:, :, :3] + background_alpha * img[glasses_y1:glasses_y2, glasses_x1:glasses_x2]

      #cv2.rectangle(img, pt1 = (x1, y1), pt2 = (x2, y2), color = (255, 0, 0), thickness = 2)

    except:
      pass

  out.write(img)

out.release()

play_video('output.mp4')